In [5]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy  as np

# Optimización hiperparámetors XGBoost

In [8]:
mi_dir = '/home/xavi/Escritorio/midirectorio/'

# Identificar características y variable objetivo
train = pd.read_csv(f'{mi_dir}DATA/train_scaffold.csv')
test = pd.read_csv(f'{mi_dir}DATA/test_scaffold.csv')
target = 'peakwavs_max'
features = [col for col in train.columns if col not in ['smiles', 'solvent', target]]

#Hacerlo con un subconjunto para que sea más rápido
train = train.sample(frac =0.3, random_state=42)
test = test.sample(frac =0.3, random_state=42)

# Dividir en conjunto de entrenamiento y prueba
x_train = train[features]
y_train = train[target]

x_test = test[features]
y_test = test[target]

In [10]:
# 2. Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'max_depth': [10, 15, 20],  # Profundidad máxima del árbol
    'learning_rate': [0.01, 0.1],  # Tasa de aprendizaje
    'n_estimators': [500, 700, 900],  # Número de árboles
    'subsample': [0.8, 1.0],  # Submuestra de datos para cada árbol
    'colsample_bytree': [1.0],  # Submuestra de características por árbol
    'gamma': [0, 0.1],  # Reducción mínima de pérdida para hacer una partición
}

# 3. Configurar Grid Search
xgb_model = XGBRegressor(tree_method='hist', objective='reg:squarederror', random_state=42)

grid_search = GridSearchCV(estimator=xgb_model,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=2, verbose=10,
                           n_jobs=-1)

# 4. Entrenar el modelo con Grid Search
grid_search.fit(x_train, y_train)

Fitting 2 folds for each of 72 candidates, totalling 144 fits


GridSearchCV(cv=2,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1],
                         'max_depth': [10, 15, 20],
                         'n_estimators': [500, 700, 900],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=10)

In [14]:
# 5. Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# 6. Evaluar el modelo con los mejores hiperparámetros
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Error Absoluto Medio (MAE) en el conjunto de prueba: {mae:.4f}")

Mejores hiperparámetros encontrados:
{'colsample_bytree': 1.0, 'gamma': 0.1, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 900, 'subsample': 0.8}
Error Absoluto Medio (MAE) en el conjunto de prueba: 31.6820
